# PWA for pγ → ΛK⁺π⁰ via `ComPWA`

PWA study on $p \gamma \to K^+ \pi^0 \Lambda$

In [ ]:
from __future__ import annotations
from IPython.display import Math
from qrules.particle import load_pdg
from ampform.io import aslatex, improve_latex_rendering

improve_latex_rendering()
particle_db = load_pdg()

In [ ]:
from qrules.particle import Particle, Spin

pgamma = Particle(
    name="pgamma",
    latex=r"p\gamma (s1/2)",
    spin=0.5,
    mass=4.101931071854584,
    charge=1,
    isospin=Spin(1 / 2, +1 / 2),
    baryon_number=1,
    parity=-1,
    pid=99990,
)
pgamma

In [ ]:
pgamma2 = Particle(
    name="pgamma2",
    latex=R"p\gamma (s3/2)",
    spin=1.5,
    mass=4.101931071854584,
    charge=1,
    isospin=Spin(1 / 2, +1 / 2),
    baryon_number=1,
    parity=-1,
    pid=99991,
)
pgamma2

In [ ]:
particle_db.add(pgamma)
particle_db.add(pgamma2)

In [ ]:
import qrules
import graphviz

reaction1 = qrules.generate_transitions(
    initial_state=("pgamma"),
    final_state=["Lambda", "K+", "pi0"],
    # allowed_intermediate_particles=["a(2)(1320)","N","Delta"],
    allowed_interaction_types=["strong"],
    formalism="canonical-helicity",
    particle_db=particle_db,
    max_angular_momentum=3,
    max_spin_magnitude=3,
    mass_conservation_factor=0,
)

In [ ]:
particle_db["Lambda"]

In [ ]:
dot_se = qrules.io.asdot(reaction1, collapse_graphs=True)
graphviz.Source(dot_se)

In [ ]:
import ampform

model_builder = ampform.get_builder(reaction1)
model_no_dynamics = model_builder.formulate()
model_no_dynamics.intensity

In [ ]:
from ampform.io import aslatex

Math(aslatex(model_no_dynamics.amplitudes))

In [ ]:
Math(aslatex(model_no_dynamics.parameter_defaults))

In [ ]:
from ampform.dynamics.builder import (
    create_non_dynamic_with_ff,
    create_relativistic_breit_wigner_with_ff,
)

model_builder.dynamics.assign("pgamma", create_non_dynamic_with_ff)
for name in reaction1.get_intermediate_particles().names:
    model_builder.dynamics.assign(name, create_relativistic_breit_wigner_with_ff)
model = model_builder.formulate()